In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Import dataset from merged dataset
data = pd.read_csv("../data/data_merged.csv", encoding='iso-8859-1', low_memory=False)

In [ ]:
data.columns

In [ ]:
data.shape

## Filter the data: 
### It will be easier to interpert *elapsed_workdays* if *TaskStatus* = Complete and exclude customer operations

In [ ]:
df = data[data['TaskStatus'] == 'Complete']
df = df[(df['AGENCY'] == 'SISTER')|(df['AGENCY'] == 'DCRA')]

In [ ]:
df.shape

In [ ]:
# Drop unrelavent variables
X = df.drop(['Unnamed: 0', 'As_Of_Date', 'permit_cap_status','Project_Description','Project_CreateDate',
             'TaskStatus', 'FlowTask_CreatedDate','FlowTask_DateUpdated', 'FlowTask_DateAccepted',
             'RC_ReviewStatus', 'RC_ReviewComments_confidential', 'Fee_Assessed','over_30','pdox',
             'est_worktime', 'QA_Date', 'QA_Date_rc1', 'DESC_OF_WORK', 'issued', 'Begin_review_dd',
             'Resubmit_rec_dd', 'rc_1_time', 'rc_time', 'FlowInstanceID', 'create_review', 'ssl',
             'RC_assignedBy_anon', 'UpdatedByEmail_anon', 'applicant_anon','owner_name_anon', 'NAME_anon',
             'OBJECTID', 'BLDG_NUM', 'SALEDATE', 'AYB', 'SALE_NUM', 'GIS_LAST_MOD_DTTM'], axis=1)

In [ ]:
X.columns

## Data Cleaning

In [ ]:
# Clean GroupName variable
# Suppress chaining warning
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
X.GroupName[X.GroupName == 'DC Water Review Supervisor'] = "DC Water Review"
X.GroupName[X.GroupName == 'DDOT Review Supervisor'] = "DDOT Review"
X.GroupName[X.GroupName == 'DOH Review Supervisor'] = "DOH Review"
X.GroupName[X.GroupName == 'Electrical Review Supervisor'] = "Electrical Review"
X.GroupName[X.GroupName == 'Elevator Review Supervisor'] = "Elevator Review"
X.GroupName[X.GroupName == 'Energy Review Supervisor'] = "Energy Review"
X.GroupName[X.GroupName == 'Fire Review Supervisor'] = "Fire Review"
X.GroupName[X.GroupName == 'Green Review Supervisor'] = "Green Review"
X.GroupName[X.GroupName == 'Mechanical Review Supervisor'] = "Mechanical Review"
X.GroupName[X.GroupName == 'Plumbing Review Supervisor'] = "Plumbing Review"
X.GroupName[X.GroupName == 'Structural Review Supervisor'] = "Structural Review"
X.GroupName[X.GroupName == 'WASA Review Supervisor'] = "WASA Review"
X.GroupName[X.GroupName == 'Zoning Review Supervisor'] = "Zoning Review"

In [ ]:
X.GroupName[X.GroupName == 'DDOE Review Supervisor'] = "DDOE Review"
X.GroupName[X.GroupName == 'DDOE SE-SW Review'] = "DDOE Review"
X.GroupName[X.GroupName == 'DDOE WSP Review'] = "DDOE Review"
X.GroupName[X.GroupName == 'DOEE FP Review'] = "DOEE Review"
X.GroupName[X.GroupName == 'DOEE GAR Review'] = "DOEE Review"
X.GroupName[X.GroupName == 'DOEE Review Supervisor'] = "DOEE Review"
X.GroupName[X.GroupName == 'DOEE SE-SW Review'] = "DOEE Review"

In [ ]:
# Aggregate 'building_construction_type' variable 
X.building_construction_type[X.building_construction_type == 'Type I - Fire-Resistive Non-combustible'] = "Type I - Fire-Resistive Non-Combustible"
X.building_construction_type[X.building_construction_type == 'Type I-A - Fire-Resistive Non-Combustible'] = "Type I - Fire-Resistive Non-Combustible"
X.building_construction_type[X.building_construction_type == 'Type I-B - Fire-Resistive Non-Combustible'] = "Type I - Fire-Resistive Non-Combustible"
X.building_construction_type[X.building_construction_type == 'TYPE II-A - Non-Combustible Construction'] = "TYPE II - Non-Combustible Construction"
X.building_construction_type[X.building_construction_type == 'TYPE II-B - Non-Combustible Construction'] = "TYPE II - Non-Combustible Construction"
X.building_construction_type[X.building_construction_type == 'TYPE III-A - Non-Combustible Exterior Walls'] = "TYPE III - Non-Combustible Exterior Walls"
X.building_construction_type[X.building_construction_type == 'TYPE III-B - Non-Combustible Exterior Walls'] = "TYPE III - Non-Combustible Exterior Walls"
X.building_construction_type[X.building_construction_type == 'TYPE V-A - Any Materials Permitted'] = "TYPE V - Any Materials Permitted"
X.building_construction_type[X.building_construction_type == 'TYPE V-B - Any Materials Permitted'] = "TYPE V - Any Materials Permitted"

Change *price* and *EYB* == 0 to N.A

In [ ]:
X['PRICE'] = X['PRICE'].apply(lambda x: None if x == 0 else x)

In [ ]:
X['EYB'] = X['EYB'].apply(lambda x: None if x == 0 else x)

Change EYB to elapsed years from construction

In [ ]:
X['EYFC'] = 2018 - X['EYB']
X = X.drop(['EYB'], axis = 1)

Evaluate the variables 'proposed_number_units' and 'existing_number_units' 
Convert to a dummy to indicate whether the num of units changing in the permit application or not: 1= change/ 0=no change

In [ ]:
num_units = X['proposed_number_units'] != X['existing_number_units']
num_units = pd.DataFrame(num_units, columns=["num_units_change"])
num_units.groupby(by="num_units_change").size()

Evaluate the variables 'existing_use_of_building' and 'proposed_use_of_building' 
Convert to a dummy to indicate whether the existing use changing in the permit application or not: 1= change/ 0=no change

In [ ]:
use = X['existing_use_of_building'] != X['proposed_use_of_building']
use = pd.DataFrame(use, columns=["use_change"])
use.groupby(by="use_change").size()

Evaluate the variables'present_gfa', 'proposed_gross_floor_area' 
Convert to a dummy to indicate whether the gfa changing in the permit application or not: 1= change/ 0=no change

In [ ]:
gfa = X['proposed_gross_floor_area'] != X['present_gfa']
gfa = pd.DataFrame(gfa, columns=["gfa_change"])
gfa.groupby(by="gfa_change").size()

In [ ]:
temp = pd.concat([use, num_units, gfa], axis=1)

Create dummy variables for all "GroupName"

In [ ]:
agencies = pd.get_dummies(X.GroupName)
agencies.columns

In [ ]:
temp = pd.concat([temp, agencies], axis=1)
X = pd.concat([X, temp], axis=1)

In [ ]:
# Drop variables
X = X.drop(['existing_use_of_building', 'proposed_use_of_building','proposed_number_stories',
            'proposed_number_units','existing_number_units', 
            'proposed_gross_floor_area','present_gfa' ,'GroupName'], axis=1)

#### Log variables have big numbers: *green_floor_area*, *PRICE*, *LIVING_GBA*, *LANDAREA*

In [ ]:
X.groupby(by="green_floor_area").size()

In [ ]:
import math
X['green_floor_area'] = X['green_floor_area'].apply(lambda x: math.log(x,10) if x > 0 else x)

In [ ]:
X.groupby(by="green_floor_area").size()

In [ ]:
X['PRICE'] = X['PRICE'].apply(lambda x: math.log(x,10) if x > 0 else x)

In [ ]:
X.groupby(by="PRICE").size()

In [ ]:
X['LIVING_GBA'] = X['LIVING_GBA'].apply(lambda x: math.log(x,10) if x > 0 else x)

In [ ]:
X.groupby(by="LIVING_GBA").size()

In [ ]:
X['LANDAREA'] = X['LANDAREA'].apply(lambda x: math.log(x,10) if x > 0 else x)

In [ ]:
X.groupby(by="LANDAREA").size()

In [ ]:
X.columns

## Build New Dataset: D

### Build Feature Matrix

In [ ]:
X.sort_values("pdox_b1_id", ascending = True)
D = X.groupby('pdox_b1_id')['alias', 'Project_Status', 'job_class', 'Ward', 'green_floor_area', 'use_type',
                            'permit_type', 'PRICE', 'QUALIFIED', 'LIVING_GBA', 'USECODE',
                            'LANDAREA', 'EYFC', 'use_change', 'num_units_change', 'gfa_change'].head(1)
D = D.reset_index()
D

In [ ]:
MAX_Review_Cycle = X.groupby('pdox_b1_id')['ReviewCycle'].max()
MAX_Review_Cycle = MAX_Review_Cycle.reset_index()
MAX_Review_Cycle = MAX_Review_Cycle.drop(['pdox_b1_id'], axis=1)

In [ ]:
D = pd.concat([D, MAX_Review_Cycle], axis=1)
D

Change *agencies* into continous variables

In [ ]:
X['CFA_Review'] = X['elapsed_workdays']*agencies['CFA Review']
X['Chinatown_Review'] = X['elapsed_workdays']*agencies['Chinatown Review']
X['DC_Water_Review'] = X['elapsed_workdays']*agencies['DC Water Review']
X['DDOE_Review'] = X['elapsed_workdays']*agencies['DDOE Review']
X['DDOT_Review'] = X['elapsed_workdays']*agencies['DDOT Review']
X['DOEE_Review'] = X['elapsed_workdays']*agencies['DOEE Review']
X['DOH_Review'] = X['elapsed_workdays']*agencies['DOH Review']
X['EISF_Review'] = X['elapsed_workdays']*agencies['EISF Review']
X['Electrical_Review'] = X['elapsed_workdays']*agencies['Electrical Review']
X['Elevator_Review'] = X['elapsed_workdays']*agencies['Elevator Review']
X['Energy_Review'] = X['elapsed_workdays']*agencies['Energy Review']
X['File_Room'] = X['elapsed_workdays']*agencies['File Room']
X['Fire_Review'] = X['elapsed_workdays']*agencies['Fire Review']

In [ ]:
X['Green_Review'] = X['elapsed_workdays']*agencies['Green Review']
X['HPRB_Review'] = X['elapsed_workdays']*agencies['HPRB Review']
X['Mechanical_Review'] = X['elapsed_workdays']*agencies['Mechanical Review']
X['NCPC_Review'] = X['elapsed_workdays']*agencies['NCPC Review']
X['PRC'] = X['elapsed_workdays']*agencies['PRC']
X['Plumbing_Review'] = X['elapsed_workdays']*agencies['Plumbing Review']
X['QA_Review'] = X['elapsed_workdays']*agencies['QA Review']
X['Structural_Review'] = X['elapsed_workdays']*agencies['Structural Review']
X['WASA_Review'] = X['elapsed_workdays']*agencies['WASA Review']
X['WMATA_Review'] = X['elapsed_workdays']*agencies['WMATA Review']
X['White_House_Review'] = X['elapsed_workdays']*agencies['White House Review']
X['Zoning_Review'] = X['elapsed_workdays']*agencies['Zoning Review']

In [ ]:
AGENCY = X.groupby('pdox_b1_id')['CFA_Review', 'Chinatown_Review', 'DC_Water_Review', 'DDOE_Review', 
                                'DDOT_Review', 'DOEE_Review', 'DOH_Review', 'EISF_Review',
                                'Electrical_Review', 'Elevator_Review', 'Energy_Review', 'File_Room',
                                'Fire_Review', 'Green_Review', 'HPRB_Review', 'Mechanical_Review',
                                'NCPC_Review', 'PRC', 'Plumbing_Review', 'QA_Review',
                                'Structural_Review', 'WASA_Review', 'WMATA_Review', 'White_House_Review',
                                'Zoning_Review'].aggregate(np.sum)
AGENCY = AGENCY.reset_index()
AGENCY = AGENCY.drop(['pdox_b1_id'], axis=1)

In [ ]:
D = pd.concat([D, AGENCY], axis=1)
D

### Build Target Matrix

In [ ]:
T = X.groupby("pdox_b1_id")["elapsed_workdays"].aggregate(np.sum)
T = T.reset_index()

In [ ]:
D = pd.concat([T, D], axis=1)
D

### Drop NA

In [ ]:
D.shape

In [ ]:
D.isnull().sum()

In [ ]:
D = D.dropna()
D.shape

#### Create Dummy Target

In [ ]:
D['elapsed_workdays'].mean()

In [ ]:
np.percentile(D.elapsed_workdays, [50, 63.8])

Create dummy target based on the mean and the median

In [ ]:
D['over_90'] = D['elapsed_workdays'].apply(lambda x: 1 if x > 90 else 0)
D.groupby(by='over_90').size()

In [ ]:
D['over_150'] = D['elapsed_workdays'].apply(lambda x: 1 if x > 150 else 0)
D.groupby(by='over_150').size()

In [ ]:
D.columns

In [ ]:
D = D.drop(['index'], axis = 1)

In [ ]:
# Save
D.to_csv("../../data/final_permit_uncode.csv", index=False)

#### Encode categorical variables

In [ ]:
D

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
CV = ['alias', 'Project_Status', 'job_class', 'use_type', 'permit_type', 'QUALIFIED',
      'use_change', 'num_units_change', 'gfa_change']
for v in CV:
    D[v] = lb.fit_transform(D[v])

In [ ]:
# Save
D.to_csv("../data/final_permit_encode.csv", index=False)